 # ch8. 데이터 준비하기: 조인, 병합, 변형

## 8.1 계층적 색인

In [5]:
import pandas as pd
import numpy as np

In [6]:
data= pd.Series(np.random.randn(9),
               index=[['a','a','a','b','b','c','c','d','d'],
                     [1,2,3,1,3,1,2,2,3]])
data

a  1    1.681181
   2   -0.627952
   3   -1.317344
b  1    0.795612
   3    0.529749
c  1    1.873408
   2    1.578623
d  2   -0.282415
   3   -0.744887
dtype: float64

In [7]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [8]:
data['b']

1    0.795612
3    0.529749
dtype: float64

In [9]:
data['b':'c']

b  1    0.795612
   3    0.529749
c  1    1.873408
   2    1.578623
dtype: float64

In [10]:
data.loc[['b','d']]

b  1    0.795612
   3    0.529749
d  2   -0.282415
   3   -0.744887
dtype: float64

In [11]:
data.loc[:,2]

a   -0.627952
c    1.578623
d   -0.282415
dtype: float64

In [14]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a','a','b','b'], [1,2,1,2]],
                    columns=[['Ohio','Ohio','Colorado'],
                            ['Green','Red','Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [15]:
frame.index.names = ['key1', 'key2']

In [16]:
frame.columns.names = ['state', 'color']

In [17]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [18]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 계층의 순서를 바꾸고 정렬하기

In [19]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [20]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [21]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 계층별 요약 통계

In [22]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [23]:
frame.sum(level='color',axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### DataFrame의 컬럼 사용하기

In [24]:
frame=pd.DataFrame({'a':range(7), 'b':range(7,0,-1),
                   'c':['one','one','one','two','two',
                       'two','two'],
                   'd':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [25]:
frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [26]:
frame.set_index(['c','d'], drop= False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [27]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 데이터 합치기

### 데이터베이스 스타일로 DataFrame 합치기

In [30]:
df1= pd.DataFrame({'key': ['b','b','a','c','a','a','b'],
                  'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [31]:
df2= pd.DataFrame({'key': ['a','b','d'],
                  'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [32]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [34]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [42]:
df3 = pd.DataFrame({'1key': ['b','b','a','c','a','a','b'],
                    'data1': range(7)})
df3

,1key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [43]:
df4 = pd.DataFrame({'rkey': ['a','b','d'],
                   'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [45]:
pd.merge(df3,df4,left_on='1key',right_on='rkey')

,1key,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [46]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [48]:
df1 =pd.DataFrame({'key': ['b','b','a','c','a','b'],
                  'data1': range(6)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [49]:
df2 =pd.DataFrame({'key': ['a','b','a','b','d'],
                  'data2':  range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [50]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [51]:
pd.merge(df1,df2,how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


### 색인 병합하기

In [57]:
left1 = pd.DataFrame({'key': ['a','b','a','a','b','c'],
                     'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [58]:
right1 = pd.DataFrame({'group_val': [3.5,7]}, index=['a','b'])
right1

,group_val
a,3.5
b,7.0


In [59]:
pd.merge(left1,right1,left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [60]:
pd.merge(left1,right1,left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [61]:
left2= pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],
                   index=['a','c','e'],
                    columns=['Ohio','Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [62]:
right2= pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],
                    index=['b','c','d','e'],
                    columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [63]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [65]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [66]:
left1.join(right1,on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


### 축 따라 이어붙이기

In [67]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [68]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [69]:
s1 = pd.Series([0,1],index=['a','b'])

In [70]:
s2= pd.Series([2,3,4], index= ['c','d','e'])

In [71]:
s3= pd.Series([5,6], index= ['f','g'])

In [72]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [73]:
pd.concat([s1,s2,s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [76]:
s4= pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [77]:
pd.concat([s1,s4],axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [78]:
pd.concat([s1,s4],axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [80]:
result= pd.concat([s1,s1,s3], keys=['one','two','three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [81]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [82]:
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


## 겹치는 데이터 합치기

In [83]:
a= pd.Series([np.nan,2.5, np.nan, 3.5, 4.5,np.nan],
            index=['f','e','d','c','b','a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [84]:
b= pd.Series(np.arange(len(a), dtype=np.float64),
            index=['f','e','d','c','b','a'])
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [85]:
b[-1]=np.nan

In [86]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [87]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [89]:
b[:2].combine_first(a[2:])

a    NaN
b    4.5
c    3.5
d    NaN
e    1.0
f    0.0
dtype: float64

In [91]:
df1= pd.DataFrame({'a':[1.,np.nan,5.,np.nan],
                  'b': [np.nan,2.,np.nan,6.],
                  'c': range(2,18,4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [92]:
df2= pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],
                  'b':[np.nan,3.,4.,6.,8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [93]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN
